<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>
<h1 align="center">Setting up Sessions</h1>

A Session is a sequence of traces representing a single session (e.g. a session or a thread). Each response is represented as it's own trace, but these traces are linked together by being part of the same session.
To associate traces together, you need to pass in a special metadata key where the value is the unique identifier for that thread.

In this tutorial we will setup sessions using OpenAI and OpenInference instrumentation.

## 1. Install Dependencies and Import Libraries

Install dependencies.

In [ ]:
!pip install "openai>=1.0.0" arize-phoenix jsonschema openinference-instrumentation-openai openinference-instrumentation opentelemetry-api opentelemetry-sdk openinference-semantic-conventions

## Configure Your OpenAI API Key and Instantiate Your OpenAI Client

Set your OpenAI API key if it is not already set as an environment variable.

In [ ]:
import os
from getpass import getpass

from openai import OpenAI

if not (openai_api_key := os.getenv("OPENAI_API_KEY")):
    openai_api_key = getpass("🔑 Enter your OpenAI API key: ")
client = OpenAI(api_key=openai_api_key)

## Instrument Your OpenAI Client

Instrument your OpenAI client with a tracer that emits telemetry data in OpenInference format. [OpenInference](https://arize-ai.github.io/open-inference-spec/trace/) is an open standard for capturing and storing LLM application traces that enables LLM applications to seamlessly integrate with LLM observability solutions such as Phoenix.

In [ ]:
from openinference.instrumentation.openai import OpenAIInstrumentor

from phoenix.otel import register

tracer_provider = register(project_name="openai-sessions-example")
OpenAIInstrumentor().instrument(tracer_provider=tracer_provider)

## Run Phoenix in the Background

Launch Phoenix as a background session to collect the trace data emitted by your instrumented OpenAI client. Note that Phoenix should be run in a container in a production environment.

In [ ]:
import phoenix as px

px.launch_app().view()

## Create a bare-bones Agent

Let's create the outline of an agent that will leverage OpenAI.

In [ ]:
import uuid

from openinference.instrumentation import using_session
from openinference.semconv.trace import SpanAttributes
from opentelemetry import trace

session_id = str(uuid.uuid4())

tracer = trace.get_tracer(__name__)


@tracer.start_as_current_span(
    name="agent", attributes={SpanAttributes.OPENINFERENCE_SPAN_KIND: "agent"}
)
def assistant(
    messages: list[dict],
    session_id: str = str,
):
    current_span = trace.get_current_span()
    current_span.set_attribute(SpanAttributes.SESSION_ID, session_id)
    current_span.set_attribute(SpanAttributes.INPUT_VALUE, messages[-1].get("content"))

    # Propagate the session_id down to spans crated by the OpenAI instrumentation
    # This is not strictly necessary, but it helps to correlate the spans to the same session
    with using_session(session_id):
        response = (
            client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=[{"role": "system", "content": "You are a helpful assistant."}] + messages,
            )
            .choices[0]
            .message
        )

    current_span.set_attribute(SpanAttributes.OUTPUT_VALUE, response.content)
    return response


messages = [{"role": "user", "content": "hi! im bob"}]
response = assistant(
    messages,
    session_id=session_id,
)
messages = messages + [response, {"role": "user", "content": "what's my name?"}]
response = assistant(
    messages,
    session_id=session_id,
)
messages = messages + [response, {"role": "user", "content": "what's 4+5?"}]
response = assistant(
    messages,
    session_id=session_id,
)